In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

In [9]:
calls_2012 = pd.read_csv('../../data/raw/Calls_for_Service_2012.csv', usecols=[0,1,6,10,12,15])
calls_2012.set_index('NOPD_Item', inplace=True)

calls_2013 = pd.read_csv('../../data/raw/Calls_for_Service_2013.csv', usecols=[0,1,6,10,12,15])
calls_2013.set_index('NOPD_Item', inplace=True)

calls_2014 = pd.read_csv('../../data/raw/Calls_for_Service_2014.csv', usecols=[0,1,9,13,17,20])
calls_2014.set_index('NOPD_Item', inplace=True)

calls_2015 = pd.read_csv('../../data/raw/Calls_for_Service_2015.csv', usecols=[0,1,9,13,17,20])
calls_2015.set_index('NOPD_Item', inplace=True)

calls_2016 = pd.read_csv('../../data/raw/Calls_for_Service_2016.csv', usecols=[0,1,9,13,17,20])
calls_2016.set_index('NOPD_Item', inplace=True)

calls_2017 = pd.read_csv('../../data/raw/Calls_for_Service_2017.csv', usecols=[0,1,9,13,17,20])
calls_2017.set_index('NOPD_Item', inplace=True)

calls_2018 = pd.read_csv('../../data/raw/Calls_for_Service_2018.csv', usecols=[0,1,9,13,17,20])
calls_2018.set_index('NOPD_Item', inplace=True)

calls = pd.concat([calls_2012, calls_2013, calls_2014, calls_2015, calls_2016, calls_2017, calls_2018])

In [10]:
target_types = [
    '100',
    '100C',
    '100F',
    '100I',
    '100X',
    '20',
    '20C',
    '20F',
    '20I',
    '20X',]

target_dispositions = ['NAT', 'RTF', 'GOA']

In [130]:
accidents = calls[calls['Type_'].isin(target_types)]
accidents = accidents[accidents['Disposition'].isin(target_dispositions)]

In [131]:
locations = accidents.Location.str.replace('[()]', '')
locations = locations.str.split(',')
lat = locations.str.get(0).astype('float')
lon = locations.str.get(1).astype('float')
accidents['lat'] = lat
accidents['lon'] = lon
accidents.drop(columns='Location', inplace=True)
accidents = accidents[accidents.lat > 1]

In [132]:
accidents.drop(columns=['Type_', 'Disposition'], inplace=True)
accidents.drop_duplicates(inplace=True)

In [133]:
roads = pd.read_pickle('../../data/interim/features/roads.pickle')

In [134]:
intersections = accidents[accidents.BLOCK_ADDRESS.str.contains('&', na=False)].BLOCK_ADDRESS.str.split('&')

In [135]:
street_1 = intersections.str.get(0).str.strip()
street_2 = intersections.str.get(1).str.strip()

In [136]:
single_street = accidents[np.logical_not(accidents.BLOCK_ADDRESS.str.contains('&', na=True))].BLOCK_ADDRESS

In [137]:
street_3 = single_street[single_street.str.split(' ', n=1).str.get(0).str.contains('X', na=False)].str.split(' ', n=1).str.get(1).str.strip()
street_4 = single_street[np.logical_not(single_street.str.split(' ', n=1).str.get(0).str.contains('X', na=True))].str.strip()

In [138]:
street_0 = pd.concat([street_1, street_3, street_4])

In [141]:
accidents['street_1'] = street_0

In [142]:
accidents['street_2'] = street_2

In [143]:
accidents

,TimeCreate,BLOCK_ADDRESS,lat,lon,street_1,street_2
NOPD_Item,,,,,,
A0015112,01/01/2012 12:51:44 AM,Canal St & Tchoupitoulas St,29.951019,-90.066144,Canal St,Tchoupitoulas St
A0018412,01/01/2012 01:05:59 AM,N Claiborne Ave & I-10 W,29.973013,-90.065489,N Claiborne Ave,I-10 W
A0018912,01/01/2012 01:07:36 AM,Calhoun St & St Charles Ave,29.932895,-90.120793,Calhoun St,St Charles Ave
A0020912,01/01/2012 01:12:55 AM,Elysian Fields Onramp & I-610 W,29.991297,-90.061089,Elysian Fields Onramp,I-610 W
A0024812,01/01/2012 01:38:08 AM,Felicity St & Rousseau St,29.927927,-90.068319,Felicity St,Rousseau St
A0025612,01/01/2012 01:42:04 AM,Chef Menteur Bridge & Downman Rd,30.009736,-90.020852,Chef Menteur Bridge,Downman Rd
A0033512,01/01/2012 02:21:53 AM,Broad Onramp & Earhart Blvd,29.954091,-90.094438,Broad Onramp,Earhart Blvd
A0033912,01/01/2012 02:24:43 AM,Dauphine St & St Ann St,29.960091,-90.065770,Dauphine St,St Ann St
A0035812,01/01/2012 02:33:54 AM,Gen De Gaulle Dr & Mac Arthur Bl,29.924015,-90.019974,Gen De Gaulle Dr,Mac Arthur Bl
